In [2]:
import numpy as np
from pyscf import fci

In [3]:
norb = 6

gamma0 = 10.84
b = 1.4

gamma = np.zeros((norb,norb))

for u in range(norb):
    for v in range(norb):
        duv = b*(np.sin(abs(u-v)*np.pi/norb)/np.sin(np.pi/norb))
        gamma[(u,v)] = 1/(1/gamma0 + duv)

print(gamma)

[[10.84        0.67012859  0.39727911  0.34575147  0.39727911  0.67012859]
 [ 0.67012859 10.84        0.67012859  0.39727911  0.34575147  0.39727911]
 [ 0.39727911  0.67012859 10.84        0.67012859  0.39727911  0.34575147]
 [ 0.34575147  0.39727911  0.67012859 10.84        0.67012859  0.39727911]
 [ 0.39727911  0.34575147  0.39727911  0.67012859 10.84        0.67012859]
 [ 0.67012859  0.39727911  0.34575147  0.39727911  0.67012859 10.84      ]]


In [5]:
#numpy.random.seed(1)
norb = 6
nelec = (3,3)

h1 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,(n+1)%norb)] = -2.5
h1 = h1 + h1.T

h0 = 0
h2 = np.zeros((norb,norb,norb,norb))
H2 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,n)] -= np.sum(gamma[n,:])
    print(gamma[n,:])
    for m in range(norb):
        h2[(n,n,m,m)] = 0.5*gamma[(n,m)]
        h0 += 0.5*gamma[(m,n)]
        
        H2[n,m] = h2[(n,n,m,m)]
    
print('H2=\n',H2,'\n')
print('h0=',h0,'\n')
print(h1)

e, fcivec = fci.direct_spin1.kernel(h1, h2, norb, nelec, nroots=400,
                                    max_space=30, max_cycle=100)
e[0] += h0

print('Ground-State Energy:')
print('energy', (e[0]/6))

[10.84        0.67012859  0.39727911  0.34575147  0.39727911  0.67012859]
[ 0.67012859 10.84        0.67012859  0.39727911  0.34575147  0.39727911]
[ 0.39727911  0.67012859 10.84        0.67012859  0.39727911  0.34575147]
[ 0.34575147  0.39727911  0.67012859 10.84        0.67012859  0.39727911]
[ 0.39727911  0.34575147  0.39727911  0.67012859 10.84        0.67012859]
[ 0.67012859  0.39727911  0.34575147  0.39727911  0.67012859 10.84      ]
H2=
 [[5.42       0.33506429 0.19863955 0.17287573 0.19863955 0.33506429]
 [0.33506429 5.42       0.33506429 0.19863955 0.17287573 0.19863955]
 [0.19863955 0.33506429 5.42       0.33506429 0.19863955 0.17287573]
 [0.17287573 0.19863955 0.33506429 5.42       0.33506429 0.19863955]
 [0.19863955 0.17287573 0.19863955 0.33506429 5.42       0.33506429]
 [0.33506429 0.19863955 0.17287573 0.19863955 0.33506429 5.42      ]] 

h0= 39.961700553487375 

[[-13.32056685  -2.5          0.           0.           0.
   -2.5       ]
 [ -2.5        -13.32056685  -2.5 

In [7]:
#numpy.random.seed(1)
norb = 6
nelec = (3,3)

h1 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,(n+1)%norb)] = -2.5
h1 = h1 + h1.T

h0 = 0
h2 = np.zeros((norb,norb,norb,norb))
H2 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,n)] -= np.sum(gamma[n,:])
    print(gamma[n,:])
    for m in range(norb):
        h2[(n,n,m,m)] = 0.5*gamma[(n,m)]
        h0 += 0.5*gamma[(m,n)]
        
        H2[n,m] = h2[(n,n,m,m)]
    
print('H2=\n',H2,'\n')
print('h0=',h0,'\n')
print(h1)

eri = np.zeros([norb,norb,norb,norb])
hc1 = np.zeros([norb,norb])
for p in range(norb):
    for s in range(norb):
        for q in range(norb):
            hc1[(p,s)] += h2[(p,q,q,s)]/norb

hc2 = np.zeros([norb,norb])
for p in range(norb):
    for q in range(norb):
        for r in range(norb):
            hc2[(q,r)] += h2[(p,q,r,p)]/norb
            
for p in range(norb):
    for q in range(norb):
        for r in range(norb):
            for s in range(norb):
                eri[(p,q,r,s)] = h2[(p,q,r,s)] - 0.5*(hc1[(p,s)] + hc2[(q,r)])
                
e, fcivec = fci.direct_spin1.kernel(h1, eri, norb, nelec, nroots=400,
                                    max_space=30, max_cycle=100)
e[0]+h0

print('Ground-State Energy:')
print('energy', e[0]/6)

#print(fci.direct_spin1.energy(h1, eri, fcivec, norb=norb, nelec=norb, link_index=None))

[10.84        0.67012859  0.39727911  0.34575147  0.39727911  0.67012859]
[ 0.67012859 10.84        0.67012859  0.39727911  0.34575147  0.39727911]
[ 0.39727911  0.67012859 10.84        0.67012859  0.39727911  0.34575147]
[ 0.34575147  0.39727911  0.67012859 10.84        0.67012859  0.39727911]
[ 0.39727911  0.34575147  0.39727911  0.67012859 10.84        0.67012859]
[ 0.67012859  0.39727911  0.34575147  0.39727911  0.67012859 10.84      ]
H2=
 [[5.42       0.33506429 0.19863955 0.17287573 0.19863955 0.33506429]
 [0.33506429 5.42       0.33506429 0.19863955 0.17287573 0.19863955]
 [0.19863955 0.33506429 5.42       0.33506429 0.19863955 0.17287573]
 [0.17287573 0.19863955 0.33506429 5.42       0.33506429 0.19863955]
 [0.19863955 0.17287573 0.19863955 0.33506429 5.42       0.33506429]
 [0.33506429 0.19863955 0.17287573 0.19863955 0.33506429 5.42      ]] 

h0= 39.961700553487375 

[[-13.32056685  -2.5          0.           0.           0.
   -2.5       ]
 [ -2.5        -13.32056685  -2.5 